In [ ]:
import sys
!{sys.executable} -m pip install awswrangler 

# S3 Select

This notebook shows how to use interact with [zgw-dbstore](https://github.com/mmgaggle/zgw-dbstore) via S3 Select using [AWS SDK for Pandas](https://github.com/aws/aws-sdk-pandas) (awswranger).

Boilerplate to load awswrangler and configure it to use the Kubernetes service FQDN for zgw-dbstore.

Kubernetes services do not provide support for wildcard prefixes, so the addressing style needs to be set to path.

In [ ]:
import awswrangler as wr
import pandas

wr.config.s3_endpoint_url = 'http://s3.default.svc.cluster.local'
wr.config.addressing_style = 'path'

# would be nice to figure out a way to switch between aws-s3 and zgw-dbstore configurations

Download some sample data sets and upload them to zgw-dbstore

In [ ]:
# CSV example #1
fhv_tripdata_url = "http://bd-dist.s3.amazonaws.com/fhv_tripdata_2019-09.csv"
df = pandas.read_csv(fhv_tripdata_url)
wr.s3.to_csv(df, "s3://mybucket/fhv_tripdata_2019-09.csv", index=False)

# CSV example #2
taxi_zone_lookup_url = "http://bd-dist.s3.amazonaws.com/taxi_zone_lookup.csv"
df = pandas.read_csv(taxi_zone_lookup_url)
wr.s3.to_csv(df, "s3://mybucket/taxi_zone_lookup.csv", index=False)

# Parquet example
fhv_tripdata_pq_url = "http://bd-dist.s3.amazonaws.com/fhv_tripdata_2022-01.parquet"
# need to figure out a way to read pq from url
# wr.s3.to_csv(df, "s3://mybucket/fhv_tripdata_2022-01.parquet", index=False)

## Read CSV file

These both work with Ceph

In [ ]:
df = wr.s3.select_query(
    sql="SELECT * FROM S3Object s where s._2='\"Brooklyn\"'",
    path="s3://mybucket/taxi_zone_lookup.csv",
    input_serialization="CSV",
        input_serialization_params={
        "FileHeaderInfo": "Use",
        "RecordDelimiter": "\r\n"
    },
    use_threads=True,
)

In [ ]:
df = wr.s3.select_query(
    sql="SELECT * FROM s3object",
    path="s3://mybucket/fhv_tripdata_2019-09.csv",
    input_serialization="CSV",
    input_serialization_params={
        "FileHeaderInfo": "IGNORE",
        "RecordDelimiter": "\r\n"
    },
    use_threads=True
)

This does not appear to work with Ceph, but works with Amazon S3

In [ ]:
df = wr.s3.select_query(
    sql="SELECT * FROM s3object",
    path="s3://mybucket/fhv_tripdata_2019-09.csv",
    input_serialization="CSV",
    input_serialization_params={
        "FileHeaderInfo": "Use",
        "RecordDelimiter": "\r\n"
    },
    use_threads=True,
)
df.head()

## Read Parquet file

This does not appear to work with Ceph

In [ ]:
df = wr.s3.select_query(
        sql="SELECT * FROM s3object",
        path="s3://mybucket/fhv_tripdata_2022-01.parquet",
        input_serialization="Parquet",
        input_serialization_params={},
        use_threads=True,
)

## Read JSON file